# Homework5

In [1]:
#import all libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cosine
from numpy.linalg import norm

## Problem 1

In [2]:
#loda the data
ratings = pd.read_csv("ml-100k/u.data", sep='\t', names=["user_id", "item_id", "rating", "timestamp"])
ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    100000 non-null  int64
 1   item_id    100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


### Convert the ratings data into a utility matrix

In [4]:
#create the utility matrix
utility_matrix = ratings.pivot(index='user_id', columns='item_id', values='rating')
utility_matrix.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### The 10 most similar users for user 1

In [5]:
#center user ratings data
centered_matrix = utility_matrix.sub(utility_matrix.mean(axis=1), axis=0)
centered_matrix.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.389706,-0.610294,0.389706,-0.610294,-0.610294,1.389706,0.389706,-2.610294,1.389706,-0.610294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.290323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.709677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.125714,0.125714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#define custom cosine similarity function
def mean_centered_cosine(v1, v2):
    mask = ~np.isnan(v1) & ~np.isnan(v2)  # Only keep dimensions where both have ratings
    if np.sum(mask) == 0:
        return 0
    v1c = v1[mask]
    v2c = v2[mask]
    dot = np.dot(v1c, v2c)
    denom = norm(v1c) * norm(v2c)
    return dot / denom if denom != 0 else 0

In [7]:
#clculate similarity between user 1 and all other users
user_id = 1
target_profile = centered_matrix.loc[user_id]
similarities = {}

for other_user in centered_matrix.index:
    if other_user == user_id:
        continue
    other_profile = centered_matrix.loc[other_user]
    sim = mean_centered_cosine(target_profile, other_profile)
    similarities[other_user] = sim

In [8]:
#find top 10 most similar users to user 1
top_similar_users = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:10]
top_user_ids = [uid for uid, sim in top_similar_users]
print(f"Top 10 most similar users to User {user_id}:\n")
print(f"{'User ID':<10}{'Cosine Similarity':>20}")
print("-" * 30)
for uid, sim in top_similar_users:
    print(f"{uid:<10}{sim:>20.4f}")

Top 10 most similar users to User 1:

User ID      Cosine Similarity
------------------------------
418                     1.0000
155                     1.0000
341                     1.0000
685                     1.0000
812                     0.9999
351                     0.9964
811                     0.9881
166                     0.9808
810                     0.9010
309                     0.8944


### The expected rating for this item for user 1

In [9]:
#find all users who have rated item 508
users_rated_508 = utility_matrix[508].dropna().index
users_rated_508 = users_rated_508[users_rated_508 != user_id]

In [10]:
similarities = {}
for other_user in users_rated_508:
    sim = mean_centered_cosine(target_profile, centered_matrix.loc[other_user])
    similarities[other_user] = sim

ratings_values = [utility_matrix.loc[uid, 508] for uid in similarities.keys()]
weights = [similarities[uid] for uid in similarities.keys()]
#Predict the rating using weighted average
if len(ratings_values) > 0 and sum(np.abs(weights)) > 0:
    predicted_rating = np.average(ratings_values, weights=weights)
    print(f"Predicted rating for user {user_id} on item 508: {predicted_rating:.2f}")
else:
    print(" Cannot predict rating.")

Predicted rating for user 1 on item 508: 3.67


## Problem 2

In [11]:
#load the data
ratings = pd.read_csv("ml-100k/u.data", sep="\t", names=["user_id", "item_id", "rating", "timestamp"])
movies = pd.read_csv("ml-100k/u.item", sep="|", encoding="latin-1", header=None, usecols=[0, 1], names=["item_id", "title"])

In [12]:
#degine target users and movie
target_users = [200, 15]
target_item = 95
movie_title = movies[movies['item_id'] == target_item]['title'].values[0]

In [13]:
#build centered user profiles
user_profiles = {}
for user in target_users:
    user_ratings = utility_matrix.loc[user]
    centered = user_ratings - user_ratings.mean()
    user_profiles[user] = centered.fillna(0)

In [19]:
for user in target_users:
    print(f"\nUser ID: {user}")
    print("User Profile (Centered Ratings):")
    print(user_profiles[user].head()) 


User ID: 200
User Profile (Centered Ratings):
item_id
1    0.967593
2   -0.032407
3    0.000000
4    0.000000
5    0.000000
Name: 200, dtype: float64

User ID: 15
User Profile (Centered Ratings):
item_id
1   -1.875
2    0.000
3    0.000
4    0.000
5    0.000
Name: 15, dtype: float64


In [14]:
#build centered item profile
item_profile = pd.Series(index=utility_matrix.columns, dtype=float)
item_profile[target_item] = utility_matrix[target_item].mean()
item_profile = item_profile.fillna(0)
centered_item_profile = item_profile - item_profile.mean()

In [15]:
#compute cosine similarity and Euclidean distance
results = {}
for user in target_users:
    user_vec = user_profiles[user]
    item_vec = centered_item_profile

    cos_sim = 1 - cosine(user_vec, item_vec)
    euclidean_dist = np.linalg.norm(user_vec - item_vec)

    results[user] = {
        'cosine_similarity': cos_sim,
        'euclidean_distance': euclidean_dist
    }

In [16]:
#output results
for user in target_users:
    print(f"User {user}:")
    print(f"Cosine Similarity   : {results[user]['cosine_similarity']:.4f}")
    print(f"Euclidean Distance  : {results[user]['euclidean_distance']:.4f}")

User 200:
Cosine Similarity   : 0.0768
Euclidean Distance  : 12.8811
User 15:
Cosine Similarity   : 0.0000
Euclidean Distance  : 14.1387
